# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

import os
import time 
import json
# Import API key
from api_keys import geoapify_key

In [11]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,edinburgh of the seven seas,-37.0676,-12.3116,17.57,81,91,9.15,SH,1708151242
1,1,blackmans bay,-43.0167,147.3167,21.07,68,19,0.89,AU,1708151243
2,2,la passe,45.5549,-0.8967,6.13,83,10,1.20,FR,1708151243
3,3,malango,-2.9627,119.9001,23.93,92,100,0.65,ID,1708151244
4,4,afaahiti,-17.7500,-149.2833,26.93,76,85,8.09,PF,1708151245
...,...,...,...,...,...,...,...,...,...,...
571,571,tranqueras,-31.2000,-55.7500,18.36,93,95,2.09,UY,1708151596
572,572,moose jaw,50.4001,-105.5344,-10.86,67,18,4.63,CA,1708151564
573,573,gazli,40.1333,63.4500,-1.02,33,100,12.02,UZ,1708151597
574,574,walvis bay,-22.9575,14.5053,15.53,82,100,1.54,NaN,1708151597


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [12]:
%%capture --no-display

# Configure the map plot
#print(city_data_df)


# Configure the map plot
humidity_map = city_data_df.hvplot.points(x='Lng', y='Lat', size = "Humidity", geo = True, tiles = "OSM", 
                                  frame_width = 800, frame_height = 600, color = "City")



# Display the map
hvplot.show(humidity_map)



### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [13]:
# Narrow down cities that fit criteria and drop any results with null values
fit_cities = city_data_df[(city_data_df["Max Temp"] >= 21) & (city_data_df["Max Temp"] <= 27) &\
                            (city_data_df["Wind Speed"] <= 4.5) & (city_data_df["Cloudiness"] <= 0)] 
                           

# Drop any rows with null values
fit_cities = fit_cities.dropna()

# Display sample data
fit_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,taoudenni,22.6783,-3.9836,21.39,13,0,4.02,ML,1708151247
48,48,tura,25.5198,90.2201,25.84,25,0,2.91,IN,1708151271
140,140,teghra,25.4833,85.9500,25.28,37,0,3.57,IN,1708151326
325,325,howrah,22.5892,88.3103,26.00,57,0,2.06,IN,1708151436
392,392,bela,25.9333,81.9833,25.67,22,0,3.58,IN,1708151477
434,434,bhanpura,24.5130,75.7469,25.87,23,0,1.00,IN,1708151509
453,453,olinda,-8.0089,-34.8553,25.85,89,0,2.57,BR,1708151520
471,471,port saint john's,-31.6229,29.5448,22.87,51,0,3.36,ZA,1708151531
476,476,cartagena,10.3997,-75.5144,26.79,83,0,2.06,CO,1708151534
544,544,new norfolk,-42.7826,147.0587,22.06,39,0,1.80,AU,1708151579


### Step 3: Create a new DataFrame called `hotel_df`.

In [14]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = fit_cities[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,taoudenni,ML,22.6783,-3.9836,13,
48,tura,IN,25.5198,90.2201,25,
140,teghra,IN,25.4833,85.9500,37,
325,howrah,IN,22.5892,88.3103,57,
392,bela,IN,25.9333,81.9833,22,
434,bhanpura,IN,24.5130,75.7469,23,
453,olinda,BR,-8.0089,-34.8553,89,
471,port saint john's,ZA,-31.6229,29.5448,51,
476,cartagena,CO,10.3997,-75.5144,83,
544,new norfolk,AU,-42.7826,147.0587,39,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
params = {
          "categories":"accommodation.hotel",
          "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params )
    
    # Convert the API response to JSON format
    name_address =  name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
taoudenni - nearest hotel: No hotel found
tura - nearest hotel: No hotel found
teghra - nearest hotel: No hotel found
howrah - nearest hotel: Sun India Guest House
bela - nearest hotel: Hotel City Palace
bhanpura - nearest hotel: No hotel found
olinda - nearest hotel: Fidji Motel
port saint john's - nearest hotel: Outback Inn
cartagena - nearest hotel: Hotel Bellamar
new norfolk - nearest hotel: Woodbridge on the Derwent
pedernales - nearest hotel: Hostal Doña Chava


,City,Country,Lat,Lng,Humidity,Hotel Name
8,taoudenni,ML,22.6783,-3.9836,13,No hotel found
48,tura,IN,25.5198,90.2201,25,No hotel found
140,teghra,IN,25.4833,85.9500,37,No hotel found
325,howrah,IN,22.5892,88.3103,57,Sun India Guest House
392,bela,IN,25.9333,81.9833,22,Hotel City Palace
434,bhanpura,IN,24.5130,75.7469,23,No hotel found
453,olinda,BR,-8.0089,-34.8553,89,Fidji Motel
471,port saint john's,ZA,-31.6229,29.5448,51,Outback Inn
476,cartagena,CO,10.3997,-75.5144,83,Hotel Bellamar
544,new norfolk,AU,-42.7826,147.0587,39,Woodbridge on the Derwent


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [17]:
%%capture --no-display

# Configure the map plot
hover_map = hotel_df.hvplot.points(x='Lng', y='Lat', size = "Humidity", geo = True, tiles = "OSM", 
                              hover_cols=["Lng","Lat","City","Humidity","Hotel Name","Country"],
                              frame_width = 800, frame_height = 600, color = "City")

# Display the map
hvplot.show(hover_map)